# CSGO皮肤价格爬虫演示(Demo)

日期 ： 2023-5-18  作者：王佳何

---

> 请确保您阅读README.md 文件后再体验本notebook

__说明__:
    
* 该文件为```CSGO_Skin_spider.py```的演示版本，主要差别在于没有采用多线程爬取数据，没有保存数据到本地文件

* 这里爬取的url也只有两个，保存在同目录demo.txt文件内

In [1]:
import requests
import re
from datetime import datetime
import pandas as pd
from threading import Thread
import os
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def GetData(urls):
    global Data
    global time
    global failed_urls
    global FinishedLength
    global NumOfFailed
    global now
    
    def CollectOneURL(url):
        
        all_data = []

        data = {}

        buff = requests.get(url)

        while buff.status_code != 200:
            sleep(3)
            buff = requests.get(url)

        soup = BeautifulSoup(buff.text, 'html.parser')
        
        # time
        data['Time'] = time

        # full name
        data['Full_name'] = soup.select('div.market-list h1')[0].get_text()
        
        # skin name
        skin_name = data['Full_name'][:data['Full_name'].index('(')]

        # quality type and kind
        a = soup.select('div[class="detail-header black"]')[0].find('p').get_text().strip().split('|')

        data['Quality'] = a[1].split(' ')[0]
        data['Type'] = a[2].split(' ')[0]
        data['Kind'] = a[-1]

        # exterior
        data['Exterior'] = soup.select_one('div.scope-btns a[class="i_Btn i_Btn_trans_bule active"]').next_sibling.strip()

        # price
        data['Price'] = soup.select_one('div.scope-btns a[class="i_Btn i_Btn_trans_bule active"]').next_sibling.next_sibling['data-price']

        # num of on sale
        data['OnSale'] = soup.select('ul.new-tab li[class="selling on"] a[href="javascript:;"]')[0].get_text()
        
        all_data.append(data)
        
        # other exterior
        for i in [(i.get_text().strip(), i.find('span')) for i in soup.select('div.scope-btns a[class="i_Btn i_Btn_trans_bule j_Goods-jump"]')]:
            if i[0] not in ['崭新出厂', '略有磨损', '久经沙场', '战痕累累', '破损不堪']:
                continue
            else:
                data = {}
                data['Time'] = time
                data['Full_name'] = skin_name + '(' + i[0] + ')'
                data['Quality'] = a[1].split(' ')[0]
                data['Type'] = a[2].split(' ')[0]
                data['Kind'] = a[-1]
                data['Exterior'] = i[0]
                data['Price'] = i[1]['data-price']
                data['OnSale'] = soup.select('ul.new-tab li[class="selling on"] a[href="javascript:;"]')[0].get_text()
                all_data.append(data)
        
        return all_data
    
    for url in urls:
        try:
            Data += CollectOneURL(url)
            #print(f'URL \033[1;32m{url}\033[0m Collected !'.center(70, ' '), end='\r')
        except:
            failed_urls.add(url)
            NumOfFailed += 1
            #print(f'URL \033[1;31m{url}\033[0m Failed......'.center(70, ' '), end='\r')
        FinishedLength += 1
        percent = round((FinishedLength / TaskLength)*100, 2)
        if FinishedLength > 10:
            ExpectedTime = int(float(((datetime.now() - now).total_seconds() / FinishedLength)*(TaskLength - FinishedLength)))
            ExpectedTime = '{}h {}min {}sec'.format(ExpectedTime // 3600, (ExpectedTime % 3600) // 60, ExpectedTime % 60)
        else:
            ExpectedTime = '......'
        
        sleep(1)
        
        print(f'Finished: \033[1;32m{percent}\033[0m%. Time to finish: \033[1;32m{ExpectedTime}\033[0m . Num of Failed: \033[1;31m{NumOfFailed}\033[0m / {FinishedLength}'.center(75, ' '), end='\r')
        
            
    return None

---

### 这里是demo.txt文件里要爬取的网址，你也可以自己打开查看:
    
* [M249（StatTrak™） | 岩浆 (战痕累累)](https://buff.163.com/goods/38760)

* [AK-47 | 血腥运动 (略有磨损)](https://buff.163.com/goods/33870)

In [3]:
pd.read_csv('./demo.txt').values

array([['https://buff.163.com/goods/38760'],
       ['https://buff.163.com/goods/33870']], dtype=object)

In [4]:
# 收集
failed_urls = set()
NumOfThreading = 10
Data = []
now = datetime.now()
time = f'{now.year}-{now.month}-{now.day} {now.hour}:{now.minute}'
FinishedLength = 0
NumOfFailed = 0

print(f'Collecting Started ! Time: {time}'.center(70, '='))

# 获取url网址, 这里不运行所有url,而是使用Demo来测试

url_list = [i[0] for i in pd.read_csv('./demo.txt').values]
TaskLength = len(url_list)

GetData(url_list)

end = datetime.now()
print(f'All Finished ! Time : {time} \033[1;32m>>>\033[0m {end.year}-{end.month}-{end.day} {end.hour}:{end.minute}'.center(81,'='))

# 保存数据
#try:
#    pd.DataFrame(Data).to_csv(f'./price_data/{time}.csv', index=False)
#    print('Data Saved Successfully !'.center(70, '='))
#except:
#    print('\033[1;31mERROR OCURED\033[0m when saving Data')

==============Collecting Started ! Time: 2023-5-31 8:58===============
========All Finished ! Time : 2023-5-31 8:58 >>> 2023-5-31 8:58=======0 / 2


### 要想查看爬取的所有数据，可以访问Data变量

In [5]:
Data

[{'Time': '2023-5-31 8:58',
  'Full_name': 'M249（StatTrak™） | 岩浆 (战痕累累) ',
  'Quality': '军规级',
  'Type': 'StatTrak™',
  'Kind': '机枪',
  'Exterior': '战痕累累',
  'Price': '7.94',
  'OnSale': '当前在售(6)'},
 {'Time': '2023-5-31 8:58',
  'Full_name': 'M249（StatTrak™） | 岩浆 (崭新出厂)',
  'Quality': '军规级',
  'Type': 'StatTrak™',
  'Kind': '机枪',
  'Exterior': '崭新出厂',
  'Price': '17.8',
  'OnSale': '当前在售(6)'},
 {'Time': '2023-5-31 8:58',
  'Full_name': 'M249（StatTrak™） | 岩浆 (略有磨损)',
  'Quality': '军规级',
  'Type': 'StatTrak™',
  'Kind': '机枪',
  'Exterior': '略有磨损',
  'Price': '9.16',
  'OnSale': '当前在售(6)'},
 {'Time': '2023-5-31 8:58',
  'Full_name': 'M249（StatTrak™） | 岩浆 (久经沙场)',
  'Quality': '军规级',
  'Type': 'StatTrak™',
  'Kind': '机枪',
  'Exterior': '久经沙场',
  'Price': '8.92',
  'OnSale': '当前在售(6)'},
 {'Time': '2023-5-31 8:58',
  'Full_name': 'M249（StatTrak™） | 岩浆 (破损不堪)',
  'Quality': '军规级',
  'Type': 'StatTrak™',
  'Kind': '机枪',
  'Exterior': '破损不堪',
  'Price': '12',
  'OnSale': '当前在售(6)'},
 {'Time': '